In [1]:
import json
import os
import itertools
import requests
import sqlparse
import re
from IPython.display import HTML, display
import tabulate
import urllib.parse

In [2]:
def get_doc_id(doc_id):
    doc_id = urllib.parse.quote(doc_id)
    url = 'https://next.obudget.org/get/'+doc_id
    print('Getting', url)
    ret = requests.get(url).json()['value']
    assert 'page_title' in ret, repr(ret)
    return ret

In [3]:
def get_table(query, headers):
    table = requests.get('https://next.obudget.org/api/query', params={'query': query}).json()
    rows = table['rows']
    ret = []
    if len(rows)>0:
        headers_ = rows[0].keys()
        for h in headers_:
            if h not in headers:
                headers.append(h)
        ret.append(headers)
        ret.extend([[x[h] for h in headers] for x in rows][:3])
    return ret + [['Total', table['total']]]

In [4]:
def questions():
    base = 'app/descriptors'
    tgt = re.compile('questions.*\.json')
    for root, dirs, fnames in os.walk(base):
        for f in fnames:
            if tgt.match(f):
                yield root.replace(base + '/', ''), os.path.join(root, f)

In [5]:
def parsed_questions():
    for q in questions():
        print(q)
        qq = json.load(open(q[1]))
        for qqq in qq:
            yield q[0], qqq

In [6]:
def parameters(q):
    p = q.get('parameters', {})
    keys = p.keys()
    values = [list(p[k].items()) for k in keys]
    return (zip(keys, prod) for prod in itertools.product(*values))

In [7]:
DOC_IDS = {
    'budget':[
        'budget/C1/2018',
        'budget/C222/2018',        
        'budget/0020/2018',
        'budget/0000/2018',
        'budget/002043/2018',
        'budget/00204301/2018',
        'budget/0020430101/2018',
    ],
    'supports': [
        'supports/0020460604/2016/עירית רחובות/הקצבות'
    ],
    'org': [
        'org/company/520039710'
    ],
    'contract-spending': [
        'contract-spending/4501218324/0033060240',
        'contract-spending/4501252882/0020400228',
    ],
    'budget-changes': [
        'national-budget-changes/2015/26-004'
    ],
    'tenders/exemptions': [
        'tenders/exemptions/31252/none'
    ]
}

In [8]:
params_re = re.compile('(:[a-z_]+)')
for kind, q in parsed_questions():
    text = q['text']
    query = q['query']
    if isinstance(query, list):
        query = ' '.join(query)
    headers = q.get('headers', [])
    doc_ids = DOC_IDS[kind]
    for doc_id in doc_ids:
        doc = get_doc_id(doc_id)
        for p in parameters(q):
            params = {}
            updated_text = q['text']
            updated_query = query
            params.update(doc)
            for name, (disp, val) in p:
                params[name] = val
                updated_text = updated_text.replace('<'+name+'>', disp)
            display(HTML('<h3>({}) {}: {}</h3>'.format(kind, doc['page_title'], updated_text)))
            ofs = 0
            while True:
                m = params_re.search(updated_query[ofs:])
                if m is None: break
                key = m.groups(0)[0][1:]
                if key in params:
                    updated_query = updated_query[:(m.start()+ofs)] + str(params[key]) + updated_query[(m.end()+ofs):]
                else:
                    print('skipping parameter', key)
                    ofs = ofs + m.start()+1
            display(HTML('<pre>{}</pre>'.format(sqlparse.format(updated_query, reindent=True, keyword_case='upper'))))
            display(HTML(tabulate.tabulate(get_table(updated_query, headers), tablefmt='html')))


('budget', 'app/descriptors/budget/questions.json')
Getting https://next.obudget.org/get/budget/C1/2018


skipping parameter json
skipping parameter json
skipping parameter json


שנה,תקציב מקורי,תקציב אחרי שינויים,ביצוע בפועל
1997,34950827000.0,42229942000.0,37875516939.0
1998,37689039000.0,45537812000.0,41255396783.0
1999,39973799000.0,48339068000.0,43656479170.0
Total,21,,


Getting https://next.obudget.org/get/budget/C222/2018


skipping parameter json
skipping parameter json
skipping parameter json


...

Total,0


skipping parameter id


Total,0


skipping parameter id


Total,0
